In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import json
from sqlalchemy import create_engine, inspect, Table, Column
import pandas as pd
from ilxutils.args_reader import read_args
from ilxutils.scicrunch_client import scicrunch
import sys
import time
from ilxutils.interlex_sql import interlex_sql
import re
from collections import defaultdict
from IPython.core.display import display
import subprocess as sb
import pickle
from pathlib import Path as p
import requests as r
from ilxutils.ttl2pd import *
import difflib
from ilxutils.tools import *

In [2]:
args = read_args(api_key= p.home() / 'keys/beta_api_scicrunch_key.txt', db_url= p.home() / 'keys/beta_engine_scicrunch_key.txt', beta=True)
#args = read_args(api_key= p.home() / 'keys/production_api_scicrunch_key.txt', db_url= p.home() / 'keys/production_engine_scicrunch_key.txt', production=True)
sql = interlex_sql(db_url=args.db_url)
sci = scicrunch(api_key=args.api_key, base_path=args.base_path, db_url=args.db_url)

In [3]:
tables = sql.show_tables()
column_name = list(tables)[0]
tables[tables[column_name].str.contains('datasets')]

,Tables_in_nif_eelg
31,community_datasets
50,datasets


In [7]:
def scope_table(df, scope):
    for column_name in list(df):
        try:
            if not df[df[column_name].str.contains(scope)].empty:
                print(column_name)
                return df[df[column_name].str.contains('jgotest')]
        except:
            continue
scope_table(c, 'Pristionchus lab')

In [24]:
terms = sql.get_terms()
print(set(terms.cid))
terms[terms.cid == 108]

{0, 1, 97, 2, 105, 73, 108, 113}


,id,ilx,orig_uid,orig_cid,uid,cid,label,definition,comment,type,version,status,display_superclass,orig_time,time
38402,38415,tmp_0138402,31699,108,31699,108,test cde 2,,,cde,1,0,1,1502124066,1502124066
38404,38417,tmp_0138404,31699,108,31699,108,TestCDE,test,,cde,1,0,1,1502217254,1502217254
38409,38422,tmp_0138409,33572,108,33572,108,jamesCDE1,,,cde,1,0,1,1502475741,1502475741
38410,38423,tmp_0138410,33572,108,33572,108,jamesCDE2,,,cde,1,0,1,1502475808,1502475808
38411,38424,tmp_0138411,33572,108,33572,108,jamesCDE3,,,cde,1,0,1,1502475996,1502475996


In [23]:
df = sql.get_table('communities')
df[df.id == 108]

,id,uid,name,description,address,portalName,url,shortName,logo,private,...,time,altPortalName,rid,verified,front_page_visible,search_name_comm_resources,search_name_more_resources,search_name_literature,mailchimp_api_key,mailchimp_default_list
73,108,31699,jgotest,jgotest,jgotest,jgotest,,jgotest,jgotest_814183.png,1,...,1498162116,None,NaN,0,0.0,None,None,None,None,None


In [ ]:
tables = sql.show_tables()
column_name = list(tables)[0]
for tablename in tables['Tables_in_nif_eelg']:
    df = sql.get_table(tablename)
    for column_name in list(df):
        try:
            if not df[df[column_name].str.contains('jgotest')].empty:
                print(tablename)
        except:
            continue

communities
communities
communities
communities
communities
notifications


In [15]:
df = sql.get_table('dataset_metadata')
df.head()

,id,datasetid,dataset_metadata_id,val


In [6]:
ds = sql.get_table('datasets')
dsf = sql.get_table('dataset_fields')
labs = sql.get_table('labs')
dft = sql.get_table('dataset_fields_templates')
c = sql.get_table('communities')

In [5]:
ds.head()

,id,uid,name,long_name,description,publications,timestamp,dataset_fields_template_id,lab_status,field_set,record_count
0,3,33352,B&B MASCIS,Beattie Bresnahan MASCIS dataset,Data from 1994-1996 MASCIS study collected at ...,,1501873020,3,pending,Laboratory,NaN
1,4,33352,BB OSU MASCIS,Beattie Bresnahan MASCIS dataset,Beattie Bresnahan OSU MASCIS dataset from 1994...,,1501875138,4,approved-internal,"Laboratory,SexSubjectGenotypTyp,AnimalID,PreIn...",NaN
2,5,53,Maryann's test data,Maryann's really awesome fabulous test data,I think the title says it all,,1502471826,4,approved-internal,"Laboratory,SexSubjectGenotypTyp,AnimalID,PreIn...",NaN
3,6,33464,fff,fff,karims test,,1502472394,6,not-submitted,None,0.0
4,10,33476,new dataset,new dataset long name,here it is,,1503068560,15,pending,"Investigator,PreInjurySurgProc,PreinjuryCond h...",NaN


In [36]:
dft.head()

,id,uid,timestamp,labid,name,active,parent_id,submitted
0,1,31699,1520564175,1,test data 1 template,0,0,0
1,2,31699,1520566591,1,test template,1,0,1
2,3,31699,1520567137,1,test 2 template,1,0,1
3,4,31699,1520567237,1,test 3 template,1,0,1
4,5,31699,1520567347,1,test 4 template,1,0,1


In [35]:
dsf.head()

,id,dataset_fields_template_id,name,termid,required,position,queryable,multi,multi_suffixes
0,1,2,Mouse ID,15029,0,0,1,0,
1,2,2,metabolite,15029,0,1,1,0,
2,3,2,Measurement,15029,0,2,1,0,
3,4,2,after treatment,15029,0,3,1,0,
4,5,2,Treatment,15029,0,4,1,0,


In [8]:
labs.head()

,id,cid,uid,name,private_description,public_description,timestamp,curated,broadcast_message
0,1,108,31699,Pristionchus lab,Ppa,Ppa,1501005989,approved,None
1,2,97,247,Jeff's Primary Lab,This is my primary lab for testing ODC-SCI,This is Jeff's primary lab for testing ODC-SCI,1501866803,approved,
2,3,97,33353,Adam's Test lab,This adam's test lab,This is Adam's Testlab,1501870636,approved,None
3,4,97,33352,Nielson Lab,Datasets for the Nielson lab,Nielson lab datasets,1501871504,approved,None
4,5,97,33464,Karim Fouad,my lab,Fouad lab,1502472078,approved,None


In [6]:
labs[labs.name.str.contains('Pristionchus lab')]

,id,cid,uid,name,private_description,public_description,timestamp,curated,broadcast_message
0,1,108,31699,Pristionchus lab,Ppa,Ppa,1501005989,approved,None


In [11]:
ds[ds.dataset_fields_template_id == 5]

,id,uid,name,long_name,description,publications,timestamp,dataset_fields_template_id,lab_status,field_set,record_count
5,14,31699,subject dataset,subject dataset,en to tre,,1507573844,5,not-submitted,"Laboratory,condition,cde 2,test,notes",0.0


In [9]:
dft[dft.labid == 1]

,id,uid,timestamp,labid,name,active,parent_id,submitted
0,1,31699,1501006000,1,template 1,0,0,0
4,5,31699,1502124075,1,template 1,0,1,1
8,9,33572,1502475708,1,template2,0,0,0
9,10,31699,1502485757,1,template 3,0,0,0
13,14,31699,1502984552,1,template 4,0,0,0
19,20,31699,1507574390,1,Template One,1,0,1
32,56,31699,1511897149,1,test 1,1,0,1
40,70,31699,1512774648,1,template-827,0,0,0
42,72,31699,1512776586,1,template-1934,0,0,0
43,73,31699,1512776858,1,template-7aa,0,0,0
